In [ ]:
# PARTE 1
# Import needed libraries
import os, time
import pandas as pd
from datetime import datetime, timedelta
from stat import * # ST_SIZE etc

In [ ]:
# PARTE 2 
# Generates list of folder contents
path = '\\\\brrio.vnet\\Backup_Azure\\Logs'
os.chdir(path)
arquivos = os.listdir()

print(arquivos[0:8])

In [ ]:
# PARTE 3
# Create list to receive .log files
lista_arquivos = list()

# Filters only files .log 
for arquivo in arquivos:
    try:
        separa = arquivo.split('.')[1]
        if separa == 'log':
            lista_arquivos.append(arquivo)
    except:
        continue
        
print(lista_arquivos[0:8])

In [ ]:
# PARTE 4
# Create temporary lists
temp = list()
lista = list()

# Loop for 
for arquivo in lista_arquivos:
    fullpath = path + '\\' + arquivo
    # Getting file information
    st = os.stat(arquivo) 
    # Convert dates
    dt_cr = time.asctime(time.localtime(st[ST_MTIME]))
    data_converte = datetime.strptime(dt_cr, '%a %b %d %H:%M:%S %Y')
    data_criacao = data_converte.date()
    # Set previous date
    data_temp = datetime.now() - timedelta(days = 1)
    data_importa = data_temp.date()
    hora_criacao =  data_converte.time()
   
   # Consider only dates larger than the previous day
    if data_criacao > data_importa:
        try:
            with open(arquivo) as f:
                conteudo = f.readlines()

            # Get path name   
            insert = conteudo[3]
            inicio = insert.find('//')
            fim = insert.find('*')
            pasta = insert[inicio:fim].replace('//prefix_name/','')                              

            # Get importation results. This is information is in the botton of the files
            sumario = conteudo[len(conteudo)-8:len(conteudo)-1]
            for cada in sumario:
                separa = cada.replace('\n','').split(':')
                indicador = separa[0]
                valor = separa[1]
                temp = pasta, arquivo, indicador, valor, data_criacao, hora_criacao
                # Insert in the list
                lista.append(temp)
        except:
            print('O arquivo %s do dia XX está com problema.' %arquivo)
            continue



print(df_erro)
print(lista[0:10])

In [ ]:
# PARTE 5
# Convert list to a dataframe 
indice = ('pasta', 'arquivo', 'indicador', 'valor', 'data_criacao', 'hora_criacao')
df = pd.DataFrame(columns = indice)

for n in lista :
    registro = ({'pasta': n[0], 'arquivo': n[1], 'indicador': n[2], 'valor': n[3], 'data_criacao': n[4], 'hora_criacao': n[5]})
    df = df.append(registro,ignore_index=True)
print(df)

In [ ]:
# PARTE 6
# Divide indicator column into one column by indicator

df['Elapsed_Time_Minutes'] = None
df['Total_Number_Transfers'] = None
df['Number_Transfers_Completed'] = None
df['Number_Transfers_Failed'] = None
df['Number_Transfers_Skipped'] = None
df['TotalBytes_Transferred'] = None
df['Final_Job_Status'] = None

for b in df.itertuples():
    df.Elapsed_Time_Minutes[b.Index] = b.valor.replace('.',',') if b.indicador == 'Elapsed Time (Minutes)' else ''
    df.Total_Number_Transfers[b.Index] = b.valor if b.indicador == 'Total Number Of Transfers' else ''
    df.Number_Transfers_Completed[b.Index] = b.valor if b.indicador == 'Number of Transfers Completed' else ''
    df.Number_Transfers_Failed[b.Index] = b.valor if b.indicador == 'Number of Transfers Failed' else ''
    df.Number_Transfers_Skipped[b.Index] = b.valor if b.indicador == 'Number of Transfers Skipped' else ''
    df.TotalBytes_Transferred[b.Index] = b.valor if b.indicador == 'TotalBytesTransferred' else ''
    df.Final_Job_Status[b.Index] = b.valor if b.indicador == 'Final Job Status' else ''
        
df